# Task1 - Age Prediction

This task is primarily concerned with regression. However, the original MRI features are perturbed in several ways. It is needed to perform outlier detection, feature selection, and other preprocessing to achieve the best result.

In [4]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr
from sklearn.ensemble import IsolationForest

### Load Data

In [5]:
X = pd.read_csv("X_train.csv")
X = X.drop('id', 1)
y = pd.read_csv("y_train.csv")
y = y.drop('id', 1)
x_test = pd.read_csv("X_test.csv")
x_test = x_test.drop('id', 1)

In [6]:
X.describe()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x822,x823,x824,x825,x826,x827,x828,x829,x830,x831
count,1118.000000,1114.000000,1117.000000,1106.000000,1.117000e+03,1128.000000,1105.000000,1127.000000,1.116000e+03,1124.000000,...,1.134000e+03,1125.000000,1098.000000,1121.000000,1120.000000,1109.000000,1115.000000,1.112000e+03,1124.000000,1091.000000
mean,10.026057,832442.859290,20585.524887,1048.958235,1.000291e+06,10.085010,597900.429955,10389.657239,9.998422e+05,785176.225858,...,1.049674e+06,-876.044006,13492.600186,10.554762,10.057767,1066.141107,10.008269,1.050199e+06,99798.480171,104903.905758
std,0.968347,0.028258,0.029051,28.430733,9.740891e+04,0.968026,0.028128,1655.843472,1.022441e+05,0.028799,...,2.839579e+04,164.585576,2519.835006,0.283844,0.982656,226.606986,1.018930,2.814210e+04,9576.128720,2768.405350
min,6.672068,832442.808579,20585.473808,1000.063783,6.800215e+05,6.984052,597900.381003,3644.074892,6.095730e+05,785176.176297,...,1.000105e+06,-1597.766964,2536.030655,10.010366,6.841039,496.007706,6.466963,1.000002e+06,73207.994891,100012.896777
25%,9.381273,832442.835941,20585.501013,1024.969967,9.360882e+05,9.470582,597900.406110,9339.537887,9.322937e+05,785176.201279,...,1.025054e+06,-975.398714,11947.954006,10.321039,9.379001,899.067501,9.325229,1.027575e+06,93416.252400,102596.190683
50%,10.000079,832442.860041,20585.524817,1047.985497,1.000557e+06,10.089601,597900.429787,10295.013382,1.001261e+06,785176.225608,...,1.049296e+06,-875.508235,13352.186179,10.554260,10.114370,1049.027077,10.005684,1.050262e+06,99802.127899,104846.235709
75%,10.664998,832442.882951,20585.550525,1073.180317,1.064617e+06,10.752707,597900.452983,11304.073469,1.068359e+06,785176.250421,...,1.074354e+06,-773.174562,14893.726023,10.792195,10.745370,1215.057985,10.658120,1.073831e+06,106400.748441,107098.669350
max,12.956099,832442.908334,20585.573514,1099.977638,1.331630e+06,12.747734,597900.480810,17347.531573,1.284804e+06,785176.276168,...,1.099771e+06,-281.030205,24815.260375,11.091050,13.530204,2122.032859,13.163113,1.099918e+06,130694.436443,109984.169649


In [7]:
x_test.describe()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x822,x823,x824,x825,x826,x827,x828,x829,x830,x831
count,727.000000,724.000000,727.000000,720.000000,7.260000e+02,738.000000,722.000000,735.000000,7.370000e+02,734.000000,...,7.260000e+02,732.000000,721.000000,725.000000,710.000000,734.000000,735.000000,7.240000e+02,726.000000,726.000000
mean,10.039015,832442.858171,20585.522048,1048.806574,1.003481e+06,10.046747,597900.433620,10326.855165,1.001574e+06,785176.226205,...,1.050193e+06,-880.981905,13402.393904,10.549199,10.040526,1059.800789,9.989603,1.048411e+06,100028.051966,104962.290638
std,1.042356,0.028952,0.028884,27.969652,9.972488e+04,1.027675,0.028687,1430.386459,9.686262e+04,0.028371,...,2.786949e+04,142.512456,2164.732346,0.285778,1.029723,210.411508,1.019917,2.935749e+04,10067.236955,2880.688742
min,6.234237,832442.808761,20585.473716,1000.117013,7.102701e+05,6.560981,597900.381334,2931.083528,6.662819e+05,785176.176301,...,1.000034e+06,-1399.750445,4024.310538,10.015383,6.522140,514.028202,6.959936,1.000218e+06,70433.222112,100035.002753
25%,9.274539,832442.834442,20585.497914,1024.738116,9.403978e+05,9.352914,597900.409501,9381.043215,9.363215e+05,785176.202904,...,1.027603e+06,-986.599727,11929.246721,10.309383,9.321506,912.043338,9.274415,1.024078e+06,93148.581708,102472.807919
50%,10.091882,832442.858015,20585.521388,1049.164447,1.006202e+06,10.073223,597900.435072,10321.053351,1.003485e+06,785176.225224,...,1.051370e+06,-876.364262,13425.916231,10.556500,10.117126,1047.546785,9.960825,1.045739e+06,99818.695142,105023.965025
75%,10.778216,832442.883733,20585.545600,1072.179216,1.070030e+06,10.754008,597900.459082,11194.055622,1.062119e+06,785176.251549,...,1.073474e+06,-786.175521,14817.701835,10.785446,10.698677,1176.783019,10.688383,1.073518e+06,106348.009270,107413.804690
max,13.138966,832442.908170,20585.573704,1099.816491,1.329045e+06,12.775692,597900.480556,14470.083785,1.303770e+06,785176.276171,...,1.099708e+06,-454.289015,19087.907988,11.082954,13.217272,2050.058057,13.604570,1.099921e+06,133791.364466,109966.906631


### Define necessary functions

In [ ]:
def custom_r2(prediction, train_data):
    """Regular r2 cost function returned as a tuple to be used with lgb"""
    labels = train_data.get_label()
    return 'r2', r2_score(labels, prediction), True


def fill_missing_values(X):
    """Fill missing values with the median of the current column"""
    for columnName in X:
        X[columnName] = X[columnName].fillna(X[columnName].median())


def outlier_mask(X):
    """Returns boolean mask of outliers"""
    # identify outliers in the training data set
    ee = IsolationForest(contamination=0.1)
    yhat = ee.fit_predict(X)
    # select all rows that are not outliers
    return yhat != -1


def feature_selection(X, y):
    """Feature selection based on the Pearson correlation coefficient."""
    features = X.iloc[:, :].columns.tolist()
    labels = y["y"].values
    correlations = {}
    squares = {}
    cubes = {}
    for key in features:
        data_temp = X[[key]]
        x1 = data_temp[key].values
        correlations[key] = pearsonr(x1, labels)[0]
        squares[key] = pearsonr(x1**2, labels)[0]
        cubes[key] = pearsonr(x1**3, labels)[0]
    print("Correlations of each feature against age")
    data_correlations = pd.DataFrame(correlations, index=['linear']).T
    data_correlations = pd.concat([data_correlations, pd.DataFrame(squares, index=['square']).T], axis=1)
    data_correlations = pd.concat([data_correlations, pd.DataFrame(cubes, index=['cube']).T], axis=1)
    return data_correlations.loc[(abs(data_correlations['linear']) >= 0.2) |
                                 (abs(data_correlations['square']) >= 0.2) |
                                 (abs(data_correlations['cube']) >= 0.2)].index

### Training and Prediction

In [ ]:
# Parameters for lightgmb
params = {
            'task': 'train',
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.5,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': 0
        }

In [ ]:
fill_missing_values(X)
fill_missing_values(x_test)

mask = outlier_mask(X)
X, y = X[mask], y[mask]

selected_features = feature_selection(X, y)

X = X[selected_features].values
x_test = x_test[selected_features].values

In [ ]:
best = 0
for i in range(100):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15)
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=500,
                    feval=custom_r2,
                    valid_sets={lgb_train, lgb_eval},
                    early_stopping_rounds=20)
    y_val_p = gbm.predict(X_val, num_iteration=gbm.best_iteration)

    result = r2_score(y_val, y_val_p)
    if result > best:
        best = result
        predictions = gbm.predict(x_test, num_iteration=gbm.best_iteration)
        ids = np.arange(0, len(predictions))

        output = pd.DataFrame({'id': ids,
                                'y': predictions})
        output.to_csv("output.csv", index=False)